In [ ]:
import sqlite3
import pandas as pd
from tqdm import tqdm
import os
from dotenv import load_dotenv

from langchain_community.utilities import SQLDatabase
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [ ]:
# Connect to SQLite database (or create it if it doesn't exist)
connection = sqlite3.connect("student_grades.db")
cursor = connection.cursor()

# Create a table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS grades (
        id INTEGER PRIMARY KEY,
        name TEXT,
        subject TEXT,
        score INTEGER,
        grade TEXT
    )
""")

# Insert some dummy data
data = [
    (1, "Aman", "Math", 95, "A"),
    (2, "Anshu", "Math", 78, "C"),
    (3, "Akshu", "History", 88, "B"),
    (4, "Rahul", "History", 92, "A"),
    (5, "Divyansh", "Science", 85, "B"),
    (6, "Nandini", "Math", 65, "D")
]

cursor.executemany("INSERT OR IGNORE INTO grades VALUES (?, ?, ?, ?, ?)", data)
connection.commit()
connection.close()

print("Database created and populated successfully!")

Database created and populated successfully!


In [ ]:
db = SQLDatabase.from_uri("sqlite:///student_grades.db")

In [19]:
schema = db.get_table_info()
print("📋 Database schema:")
print(schema)

📋 Database schema:

CREATE TABLE grades (
	id INTEGER, 
	name TEXT, 
	subject TEXT, 
	score INTEGER, 
	grade TEXT, 
	PRIMARY KEY (id)
)

/*
3 rows from grades table:
id	name	subject	score	grade
1	Aman	Math	95	A
2	Anshu	Math	78	C
3	Akshu	History	88	B
*/


In [ ]:
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")


llm = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0.1
)

# Step 5: Define the SQL generation prompt
prompt = ChatPromptTemplate.from_template("""
You are a senior data analyst and SQL expert.

Given the database schema below, write a correct SQL query
that answers the user's question.

Rules:
- Use only the tables and columns in the schema
- Do NOT explain anything
- Return ONLY the SQL query

Schema:
{schema}

Question:
{question}
""")

# Step 6: Create the LangChain pipeline
sql_chain = (
    prompt
    | llm
    | StrOutputParser()
)

In [20]:
import pandas as pd

def ask_question(question: str):
    """
    Generate SQL from a natural language question using the LLM,
    execute it on the database, and display the result as a DataFrame.
    """
    try:
        # Generate SQL query
        sql_query = sql_chain.invoke({"schema": schema, "question": question}).strip()
        print("🧠 Generated SQL:")
        print(sql_query)

        # Execute the query
        result = db.run(sql_query)

        # Convert result to a DataFrame for better display
        if result:
            df = pd.DataFrame(result)
            print("\n📈 Result:")
            display(df)
        else:
            print("\n📈 Result: No rows returned.")

    except Exception as e:
        print(f"❌ Error: {e}")

# Example usage:
question = "Who scored the highest in Math?"
ask_question(question)


🧠 Generated SQL:
SELECT name
FROM grades
WHERE subject = 'Math'
ORDER BY score DESC
LIMIT 1;
❌ Error: DataFrame constructor not properly called!
